In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
 
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
 
url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"
 
r = requests.get(url, headers=headers)
r.status_code
 
# BeautifulSoup()으로 웹페이지 분석하기
soup = BeautifulSoup(r.text, 'html.parser')
# 선수들의 정보가 담긴 태그와 클래스 찾기
player_info = soup.find_all('tr', class_ = ['odd','even'])
 
# 7개 정보(이름, 나이, 소속팀, 몸값 등등)를 담을 빈 리스트 만들기
number = []
name = []
position = []
age = []
nation = []
team = []
value = []
 
# player_info에서 'td' 태그만 모두 찾기
for info in player_info:
    player = info.find_all("td")
# 해당 정보를 찾아서 각 리스트에 추가하기
    number.append(player[0].text)
    name.append(player[3].text)
    position.append(player[4].text)     # 들여쓰기 주의!
    age.append(player[5].text)
    nation.append(player[6].img['alt'])  # 태그의 특정 속성 값을 뽑아내는 코드, 딕셔너리 문법 사용( 속성 : key, 값 : value)
    team.append(player[7].img['alt'])
    value.append(player[8].text.strip())

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

player_list=[]
    
# url의 page 부분을 변수로 처리하여 반복문 실행
for i in range(1,3):
    url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}"

    r = requests.get(url, headers=headers)
    r.status_code

    # BeautifulSoup()으로 웹페이지 분석하기
    soup = BeautifulSoup(r.text, 'html.parser')
    # 선수들의 정보가 담긴 태그와 클래스 찾기
    player_info = soup.find_all('tr', class_ = ['odd','even'])

    # player_info에서 'td' 태그만 모두 찾기
    for info in player_info:
        player = info.find_all("td")
    # 해당 정보를 찾아서 각 리스트에 추가하기
        number = player[0].text 
        name = player[3].text 
        position = player[4].text 
        age = player[5].text 
        nation = player[6].img['alt'] 
        team = player[7].img['alt'] 
        value = player[8].text.strip()
        # 데이터에서 숫자만 가져오기 (인덱싱)
        value = value[1:-1] # ex. €160.00m 에서 €이 [0]이고 m이 [-1] 
        
        player_list.append([number, name, position, age, nation, team, value])
    
    time.sleep(1)  # 1페이지 당 sleep
        
    # pd.DataFrame()으로 저장하기
   
    
    df = pd.DataFrame(player_list, 
                   columns=['number', 'name', 'position', 'age', 'nation', 'team', 'value'])
   
# 저장된 파일 불러오기
# pd.read_csv('transfermarkt25.csv')

In [12]:
# 데이터 전처리(pre-processing)
df['value'] = df['value'].str.replace('€','')
df['value'] = df['value'].str.replace('m','').astype('float')
df['number'] = df['number'].astype('int64')
df['age'] = df['age'].astype('int64')
# df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   number    50 non-null     int64  
 1   name      50 non-null     object 
 2   position  50 non-null     object 
 3   age       50 non-null     int64  
 4   nation    50 non-null     object 
 5   team      50 non-null     object 
 6   value     50 non-null     float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.9+ KB


In [13]:
# 컬럼 생성 (한화 표시)
df['시장가치(억)'] = df['value']*13
df.head()

,number,name,position,age,nation,team,value,시장가치(억)
0,1,Erling Haaland,Centre-Forward,24,Norway,Manchester City,200.0,2600.0
1,2,Vinicius Junior,Left Winger,24,Brazil,Real Madrid,200.0,2600.0
2,3,Lamine Yamal,Right Winger,17,Spain,FC Barcelona,180.0,2340.0
3,4,Jude Bellingham,Attacking Midfield,21,England,Real Madrid,180.0,2340.0
4,5,Kylian Mbappé,Centre-Forward,26,France,Real Madrid,160.0,2080.0


In [14]:
# 컬럼 삭제 df.drop(columns=['칼럼이름'])
df.drop(columns=['value'], inplace = True)  # inplace = True로 해야 저장됨!
df.head()

,number,name,position,age,nation,team,시장가치(억)
0,1,Erling Haaland,Centre-Forward,24,Norway,Manchester City,2600.0
1,2,Vinicius Junior,Left Winger,24,Brazil,Real Madrid,2600.0
2,3,Lamine Yamal,Right Winger,17,Spain,FC Barcelona,2340.0
3,4,Jude Bellingham,Attacking Midfield,21,England,Real Madrid,2340.0
4,5,Kylian Mbappé,Centre-Forward,26,France,Real Madrid,2080.0


In [15]:
df.describe()

,number,age,시장가치(억)
count,50.00000,50.00000,50.000000
mean,25.50000,24.32000,1277.900000
std,14.57738,2.67597,477.941045
min,1.00000,17.00000,845.000000
25%,13.25000,23.00000,975.000000
50%,25.50000,24.50000,1040.000000
75%,37.75000,26.00000,1430.000000
max,50.00000,31.00000,2600.000000


In [16]:
# df[칼럼 이름].mean()
# 나이평균 구하기
df['age'].mean()
# 몸값 합계 구하기 sum()
df['시장가치(억)'].sum()

63895.0

In [17]:
# 선수들이 속한 가장 많은 나라는? 최빈값 mode()
df['nation'].mode()

0    England
1     France
Name: nation, dtype: object

In [18]:
# 국적이 England인 선수들은?
df[df['nation']=='England']

,number,name,position,age,nation,team,시장가치(억)
3,4,Jude Bellingham,Attacking Midfield,21,England,Real Madrid,2340.0
5,6,Bukayo Saka,Right Winger,23,England,Arsenal FC,1950.0
8,9,Phil Foden,Right Winger,24,England,Manchester City,1820.0
9,10,Cole Palmer,Attacking Midfield,22,England,Chelsea FC,1690.0
12,13,Declan Rice,Central Midfield,26,England,Arsenal FC,1430.0
17,18,Harry Kane,Centre-Forward,31,England,Bayern Munich,1170.0
41,42,Trent Alexander-Arnold,Right-Back,26,England,Liverpool FC,975.0


In [19]:
# 데이터를 그룹으로 묶어 분석
df.groupby('nation')

In [20]:
g = df.groupby('nation')
g.size()

nation
Argentina      5
Brazil         5
Colombia       1
Croatia        1
Ecuador        1
England        7
France         7
Georgia        1
Germany        3
Hungary        1
Italy          2
Netherlands    1
Nigeria        1
Norway         2
Portugal       2
Spain          6
Sweden         2
Uruguay        2
dtype: int64

In [21]:
g.count()

,number,name,position,age,team,시장가치(억)
nation,,,,,,
Argentina,5,5,5,5,5,5
Brazil,5,5,5,5,5,5
Colombia,1,1,1,1,1,1
Croatia,1,1,1,1,1,1
Ecuador,1,1,1,1,1,1
England,7,7,7,7,7,7
France,7,7,7,7,7,7
Georgia,1,1,1,1,1,1
Germany,3,3,3,3,3,3


In [22]:
# 수치형 데이터 총합 알아보기
g['시장가치(억)'].sum()

nation
Argentina       5200.0
Brazil          6955.0
Colombia        1105.0
Croatia          975.0
Ecuador         1040.0
England        11375.0
France          7865.0
Georgia         1105.0
Germany         4615.0
Hungary          975.0
Italy           2015.0
Netherlands     1040.0
Nigeria          975.0
Norway          4030.0
Portugal        1950.0
Spain           8190.0
Sweden          1950.0
Uruguay         2535.0
Name: 시장가치(억), dtype: float64

In [ ]:
# 선수들의 몸값의 합이 큰 클럽별로 정렬해서 보여주기
result = df.groupby('team')
result['시장가치(억)'].sum().sort_values(ascending=False)